### Jackie's Regex Attempt 
I used Sagar's file, "SagarsCleaningAttempt.csv" for my starting point. I created a column 'Test_6' that shows changes from previous attempts.
The code below cleans the following:

**remove leading/trailing parentheses plus words inside, ex: "some stuff) food" or "food (some stuff"

**remove complete parentheses plus words inside, ex: "(some stuff) food" or "food (some stuff) or "food (some stuff) food"

**remove dashes only at beginning/end of sentence, ex: " - some food" or "some food -"

**remove unicode characters

**remove the word 'of' and words before it, ex: "dash of salt"

In [21]:
import pandas as pd
import re
from string import digits

# Reading csv file:
#regex_df = pd.read_csv('./SagarsCleaningAttempt.csv')    #, delimiter=',', encoding="utf-8")
regex_df = pd.read_csv('./80000_90000_Clean.csv') 

small_df = regex_df.copy()
#small_df = small_df[0:500]

#Add a new column, Regex, and fill it with everything from After column
small_df['Regex'] = small_df['after']

small_df['Regex'] = small_df['Regex'].astype(str)

#small_df.drop(small_df.index[[2295]], inplace=True)
#small_df.drop(small_df.index[[7764]], inplace=True)
#small_df.drop(small_df.index[[7765]], inplace=True)
#small_df.drop(small_df.index[[7766]], inplace=True)

#First, get rid of annoying unicode characters
for i, row in small_df.iterrows():
    #make an array of ingredients for the current row
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        s = ingr
        s = s.decode('unicode_escape').encode('ascii','ignore') # remove annoying characters like ½
        s = s.strip() # remove any leading/trailing whitespace
        #print(s)
        arr[j] = s
        
    small_df.set_value(i, 'Regex', ",".join(arr)) # put back into dataframe

#delete the '-' sign at the beginning of the string
pattern = r'(^\s*\-{1,})'
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(pattern, ingr)):
            #print("match1")
            dash, rest = re.split(r'^\-', ingr)
            new_str = rest.strip() #remove leading/trailing whitespace
            #print(new_str)
            arr[j] = new_str
            
    small_df.set_value(i, 'Regex', ",".join(arr))
        
#delete the '-' sign at the end of the string
pattern = r'(\s*\-{1,})$'
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(pattern, ingr)):
            #print("match2")
            rest, dash = re.split(r'\-', ingr, 1)
            new_str = rest.strip() #remove leading/trailing whitespace
            #print(new_str)
            arr[j] = new_str
            
    small_df.set_value(i, 'Regex', ",".join(arr))
        
        
#delete parentheses (at beginning) with any words, nums, or dashes in them: "(something) blah blah"
pattern = r'^(\()[ \w\-\'\!\;\.\"\&\]\[\/0-9]*(\))'
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(pattern, ingr)):
            #print("match5")
            before_p, parenth, rest = re.split(r'[(\()(\))]', ingr, 2)
            new_str = rest.strip() #remove leading/trailing whitespace
            #print(new_str)
            arr[j] = new_str
            
    small_df.set_value(i, 'Regex', ",".join(arr))

#delete parentheses (at end) with any words, nums, or dashes in them: "blah blah (something)"
pattern = r'(\()[ \w\-\'\!\;\.\"\&\]\[\/0-9]*(\))$'
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(pattern, ingr)):
            #print("match6")
            rest, before_p, after_p, space = re.split(r'(\()[ \w\-\'\!\;\.\"\&\]\[\/0-9]*(\))$', ingr)
            new_str = rest.strip() #remove leading/trailing whitespace
            #print(new_str)
            arr[j] = new_str
            
    small_df.set_value(i, 'Regex', ",".join(arr))
        
#delete one open parenthesis (at beginning) with this case: "something) blah blah" 
pattern = r'^[ \w\-\'\!\;\.\"\&\]\[\/0-9]*(\))'
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(pattern, ingr)):
            #print("match3")
            #print(ingr)
            before_p, parenth, rest = re.split(r'^[ \w\-\'\!\;\.\"\&\]\[\/0-9]*(\))', ingr)
            new_str = rest.strip() #remove leading/trailing whitespace
            #print(new_str)
            arr[j] = new_str
            
    small_df.set_value(i, 'Regex', ",".join(arr))

#delete one open parenthesis (at end) with this case: "blah blah (something"
pattern = r'(\()[ \w\-\'\!\;\.\"\&\]\[\/0-9]*$'
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(pattern, ingr)):
            #print("match4")
            rest, parenth, after_p = re.split(r'(\()', ingr, 1)
            new_str = rest.strip() #remove leading/trailing whitespace
            #print(new_str)
            arr[j] = new_str
            
    small_df.set_value(i, 'Regex', ",".join(arr))


#get rid of extra ')' at the end: "food stuff extra symbol at end)
pattern = r'(\))$'
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(pattern, ingr)):
            #print("match')'")
            rest, parenth, space = re.split(r'(\))$', ingr)
            new_str = rest.strip() #remove leading/trailing whitespace
            #print(new_str)
            arr[j] = new_str
            
    small_df.set_value(i, 'Regex', ",".join(arr))


#delete parentheses (in the middle) with any words, nums, or dashes in them: "blah (something) blah"
pattern = r'(\()[ \w\-\'\!\;\.\"\&\]\[\/0-9]*(\))'
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(pattern, ingr)):
            #print("match7")
            print(ingr)
            print(i)
            rest1, parenth, rest2 = re.split(r'[(\()(\))]', ingr)
            new_str = (rest1 + rest2).strip() #remove any leading/trailing whitespace
            #print(new_str)
            arr[j] = new_str
            
    small_df.set_value(i, 'Regex', ",".join(arr))
        
#Get rid of the word 'of' and anything before it: "an ounce of milk"
pattern = r'\bof\b'
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(pattern, ingr)):
            #print("match8")
            #print(ingr)
            dash, rest = re.split(r'\bof\b', ingr, 1)
            new_str = rest.strip() #remove leading/trailing whitespace
            #print(new_str)
            arr[j] = new_str
            
    small_df.set_value(i, 'Regex', ",".join(arr))


#Get rid of any stray ';' symbols in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'\;', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split(';'))
            new_str = ' '.join(new_str.split())
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#replace '&amp' with just and
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'\&', ingr)):
            #print(ingr)
            new_str = ingr.replace("&amp", " and ")
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
            
#remove any stray dashes '-'
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'\-', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('-'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
            
    small_df.set_value(i, 'Regex', ",".join(arr)) 
            
#Get rid of any stray '.' symbols in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'\.', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('.'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray '/' symbols in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'\/', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('/'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray '_' symbols in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'\_', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('_'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray 'tbsp' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'tbsp', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('tbsp'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray 'tsp' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'tsp', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('tsp'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))

#Get rid of any stray 'lbs' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'lbs', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('lbs'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))    

#Get rid of any stray 'lb' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'lb', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('lb'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray 'oz' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'oz\s', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('oz'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray 'kg' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'kg', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('kg'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))

#Get rid of any stray g for grams at the beginning of the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'^\d*g\s', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('g', 1))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray 'ml' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'^\d*\s*ml\s', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('ml', 1))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray 'fl' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'^\d*\s*fl\s', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('fl', 1))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray 'mm' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'^\d*\s*mm\s', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('mm', 1))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray 'x' at the beginning of the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'^\s*x\s', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('x', 1))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray numbers in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'\d+', ingr)):
            #print(ingr)
            new_str = ingr.translate(None, digits)
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray 'cups' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'cups', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('cups'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    
#Get rid of any stray 'cup' in the ingr
for i, row in small_df.iterrows():
    arr = row['Regex'].split(",")
    for j, ingr in enumerate(arr):
        if(re.search(r'cup', ingr)):
            #print(ingr)
            new_str = ' '.join(ingr.split('cup'))
            new_str = ' '.join(new_str.split())
            #print(new_str)
            arr[j] = new_str
    
    small_df.set_value(i, 'Regex', ",".join(arr))
    

#what about words stuck together? like saltsalt
    
print("Done!")

small_df.to_csv('80000_90000_Clean.csv', sep=',', encoding='utf-8')


achiote (annatto) seeds
365
large crispin (matsu) apple or a golden delicious apple
390
shelled fresh fava beans (from about pods) or frozen fava beans
408
eggplants pcs (cut into small pieces pcs about slices) than taking the inside out
1116
plus (or more) vegetable broth
1839
lb. (about medium) golden beets
1890
x x pan or individual dessert (my dessert cups pictured above hold capacity). if you have a slightly larger or smaller pan
1927
of collards (about pounds) or two bags chopped collards
2091
dashi (see note above) or vegetable broth
2138
ancho (or regular) chili powder
2198
scotch bonnet (deseeded) - optional if you want more heat
2467
jars (each oz.) williams-sonoma roasted
3549
bottles (each ml) cabernet sauvignon
3619
1-quart wide-mouthed jar (with a lid) in glass or clear plastic
3798
or (each oz.) fire-roasted
3805
shelled fresh fava beans (from about pods) or frozen fava beans
4156
sheets (about lb.) phyllo dough
4215
english (hothouse) cucumbers
4416
english (hothouse) c

['', ')', ' something blah']


### LDA Analysis

In [31]:
import pandas as pd
from __future__ import print_function
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer


lda_df = pd.read_csv('./Clean_Recipes.csv') 

lda_small = lda_df.copy()
#lda_small = lda_small[0:50]
lda_small['Regex'] = lda_small['Regex'].astype(str)

dataset = []
for i, row in lda_small.iterrows():
    dataset.append(row['Regex'])
    
#print(dataset)
    
n_features = 2000
n_topics = 20
n_top_words = 8

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    

print("Extracting tf features for LDA...")

tf_vectorizer = TfidfVectorizer(stop_words={'english','of','for','the'}, min_df=0.1, max_df=1.0)
#tf_vectorizer = CountVectorizer(max_df=1, min_df=0, max_features=n_features, stop_words='english')

tf = tf_vectorizer.fit_transform(dataset)

print("Fitting LDA model ...")
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=100,
                                learning_method='online',
                                learning_offset=50.)

lda.fit(tf)

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf features for LDA...
Fitting LDA model ...

Topics in LDA model:

Topic #0:
powder garlic pepper onion black sauce salt water

Topic #1:
vinegar oil olive pepper red sugar salt black

Topic #2:
cheese pepper onion eggs butter garlic salt milk

Topic #3:
eggs flour sugar butter powder salt milk vanilla

Topic #4:
garlic onion pepper oil olive butter salt sugar

Topic #5:
sauce garlic pepper onion oil sugar salt butter

Topic #6:
lemon juice garlic sugar butter pepper salt oil

Topic #7:
cream butter sugar eggs salt pepper onion milk

Topic #8:
juice sugar salt butter pepper oil garlic water

Topic #9:
all purpose flour sugar butter salt vanilla eggs

Topic #10:
black pepper salt oil olive garlic onion butter

Topic #11:
olive oil salt pepper butter sugar garlic eggs

Topic #12:
red pepper onion olive oil garlic tomatoes black

Topic #13:
water sugar salt flour butter oil pepper onion

Topic #14:
vanilla sugar milk cream butter eggs salt cheese

Topic #15:
chicken pepper oni